#Dogs vs. Cats Updated

This Kernel will expand on my previous Kernel that used a Convolutional Neural Network (CNN) to predict pictures of Dogs and Cats. Most of the code that reads in and preprocesses the data was discussed in my previous kernel (https://www.kaggle.com/kasmithh/dogs-vs-cats) and is based on code from the following tutorial (https://www.youtube.com/watch?v=gT4F3HGYXf4&t=554s). After reading in and preprocessing the data, this Kernel will address the overfitting proplem that we had in the previous Kernel by using dropout layers, more convolutional layers and implementing a batch size during the training process. 

In [ ]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, MaxPool2D, Conv2D, Flatten
import cv2
from random import shuffle
import matplotlib.pyplot as plt

import os

img_path = '../input/train'
IMG_SIZE = 100

In [ ]:
def label_img(img):
    word_label = img.split('.')[0]
    if word_label == 'cat':
        return [1,0]
    if word_label == 'dog':
        return [0,1]
    

In [ ]:
def create_train_data():
    training_data = []
    for img in os.listdir(img_path):
        label = label_img(img)
        path = os.path.join(img_path,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
        shuffle(training_data)
    return training_data

In [ ]:
train = create_train_data()

In [ ]:
X = np.array([i[0] for i in train])
X = X.reshape(-1,100,100,1)
X = X/255
Y = np.array([i[1] for i in train])

##Addressing Overfitting

In order to address overfitting and improve the model from the previous Kernel we will use a series of techniques that include adding more convolutional layers, dropout layers, and specifying a batch size during the training process. Dropout layers set a specified rate of inputs to 0 during the training process or in other words nodes of the next work are "dropped out". This allows the model to generalize easier instead of just learning the training data. More on dropout can be found here: (https://keras.io/layers/core/#dropout). Batch size determines how many samples from the training set will be used to train the model at a time. So in our case with a batch size of 32, 32 samples from the training set will be used to train the model at a time. (https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network)

The code for creating the CNN should look very similar to the previous model, but this time the model includes more convolutional layers and a couple of dropout layers. Each dropout layer is set to 30% meaning that 30% of the nodes will be set to 0.

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size = (3,3), input_shape=(100,100,1), activation = 'relu'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(Dropout(0.30))
model.add(MaxPool2D(2,2))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(Dropout(0.30))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.30))
model.add(Dense(2, activation = 'softmax'))
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy', 'mae'])

During the fitting process is where we establish our batch size of 32.

In [ ]:
Fit = model.fit(X, Y, epochs = 5, batch_size = 32, validation_split = 0.30)

As we can see in the graphs below by implementing dropout layers, a batch size during training, and more convolutional layers, the model is preforming about as well as the previous model in terms of validation accuracy and loss, but is not overfitting to the training data. Validation accuracy and loss are a lot closer to that of the training data this time unlike in the first model where the training accuracy and loss were very different than that of the validation accuracy and loss.

In [ ]:
plt.plot(Fit.history['loss'])
plt.plot(Fit.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()

In [ ]:
plt.plot(Fit.history['acc'])
plt.plot(Fit.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()